In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
WORK_DIR = '/content/gdrive/MyDrive/Colab_Notebooks/DFC'
DATA_PATH = WORK_DIR + '/Data/data_fusion_train.parquet'

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords 

In [ ]:
%%time
use_threads=False
df = pd.read_parquet(DATA_PATH)

CPU times: user 24.6 s, sys: 4.6 s, total: 29.2 s
Wall time: 33.2 s


In [ ]:
%load_ext google.colab.data_table
df

,receipt_id,receipt_dayofweek,receipt_time,item_name,item_quantity,item_price,item_nds_rate,category_id,brands
0,7,5,09:50,Флуконазол капс 150мг №1 Вертекс,1.000,7,-1,-1,вертекс
1,11,6,20:34,"Молоко 3,2%,шт",2.000,8,2,78,None
2,39,4,11:28,"Борщ Станичный с тушенкой, 103 ккал (завод, з/у)",1.000,6,1,-1,None
3,39,4,11:28,"Компот из изюма, 114 ккал",1.000,4,1,71,None
4,39,4,11:28,"Макаронные изделия отварные (масло сливочное),...",1.000,4,1,71,None
...,...,...,...,...,...,...,...,...,...
45729304,10016159,4,12:39,"Карам.""Ломтик""апел/лимон/грейфр. 1кг Яшкино",0.283,10,-1,-1,None
45729310,10016159,4,12:39,СМЕТАНА 20% Залесский Фермер 200 г. пл.ст.,1.000,7,6,-1,залесский фермер
45729312,10016159,4,12:39,Слива Черная Пантера 1кг,0.439,11,-1,-1,None
45729314,10016159,4,12:39,"Хлеб ""Заварной"" ржано-пшен. 0,4г нарезка Берез.",1.000,8,-1,-1,None


In [ ]:
df.count()

receipt_id           26094077
receipt_dayofweek    26094077
receipt_time         26094077
item_name            26094077
item_quantity        26094077
item_price           26094077
item_nds_rate        26094077
category_id          26094077
brands                6801308
dtype: int64

In [ ]:
df['category_id'].unique(), df['category_id'].unique().shape 

(array([ -1,  78,  71,  70,  84,  69,  68,  40, 203,  79,   7, 117,  80,
         83,  53,  73,  81,   0,  12,  85,  77, 145,   2, 204, 139,  38,
         74, 130,  75,  49,  45, 133,  76,  82,  19,  43,  51,  61, 177,
        118,  92,  36,  30, 167,  66,  52, 107,  37,   3,  72,  62,  50,
        120,  42, 150,  57,   6, 140, 101, 163,  20, 103,   4,  31,  67,
         27,  29, 114, 102, 115,  26,  35,  39,  13,   9, 128,  60,  41,
        138,  11, 100,  24,  96, 109, 106,  56, 143, 105,  90,   1, 108,
        164,  55, 111,  58,  54,  97,  46, 121]), (99,))

In [ ]:
df.groupby('category_id').count()

,receipt_id,receipt_dayofweek,receipt_time,item_name,item_quantity,item_price,item_nds_rate,brands
category_id,,,,,,,,
-1,19610851,19610851,19610851,19610851,19610851,19610851,19610851,6156543
0,92280,92280,92280,92280,92280,92280,92280,18341
1,226,226,226,226,226,226,226,209
2,15502,15502,15502,15502,15502,15502,15502,19
3,12159,12159,12159,12159,12159,12159,12159,6824
...,...,...,...,...,...,...,...,...
164,1388,1388,1388,1388,1388,1388,1388,0
167,2064,2064,2064,2064,2064,2064,2064,0
177,11764,11764,11764,11764,11764,11764,11764,1391


In [ ]:
?df.where

In [ ]:
df['item_name'].head(10)

0                      Флуконазол капс 150мг №1 Вертекс
1                                        Молоко 3,2%,шт
2      Борщ Станичный с тушенкой, 103 ккал (завод, з/у)
3                             Компот из изюма, 114 ккал
4     Макаронные изделия отварные (масло сливочное),...
6                  Филе Горизонт (филе птицы), 218 ккал
10                         3  Бумага Упак 1/Прочи/В асс
11                         4  Бумага Упак 1/Прочи/В асс
12                          Брюки спорт мал SAMO M-1016
13                                    Бюстгальтер БН137
Name: item_name, dtype: object

In [ ]:
#train = pd.read_parquet('train_1_2_extra.parquet')
train = df[df.category_id != -1].drop_duplicates('item_name')

In [ ]:
train.shape

(48225, 9)

In [ ]:
train.head(10)

,receipt_id,receipt_dayofweek,receipt_time,item_name,item_quantity,item_price,item_nds_rate,category_id,brands
1,11,6,20:34,"Молоко 3,2%,шт",2.0,8,2,78,None
3,39,4,11:28,"Компот из изюма, 114 ккал",1.0,4,1,71,None
4,39,4,11:28,"Макаронные изделия отварные (масло сливочное),...",1.0,4,1,71,None
17,56,5,11:42,Кофе Капучино Большой Эден 18,1.0,12,1,70,None
40,105,3,01:53,Хлеб на СЫВОРОТКЕ 350г,1.0,7,-1,84,None
53,122,0,11:46,Сосиска в тесте с сыром 1шт ГЕ,3.0,7,2,84,None
58,129,3,15:17,ЛанчБаскет 5 за 300: 2 шт ОРИГ Стрипсы кур фил,1.0,8,2,69,None
62,129,3,15:17,Станд Картофель фри,2.0,8,6,69,None
63,129,3,15:17,Хот-дог Куриный СБ,1.0,9,2,69,None
64,129,3,15:17,Чизбургер с луком СБ,1.0,9,2,68,None


In [ ]:
train['item_name'].apply(len).sort_values(ascending=True)

1394927       0
3038750       1
98123         1
21261954      1
564744        1
           ... 
3987102     128
4712512     128
1406157     128
1161697     128
1695478     128
Name: item_name, Length: 48225, dtype: int64

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stop = stopwords.words('russian')
tfidf = TfidfVectorizer(stop_words=stop, max_features=1000)
X_train = tfidf.fit_transform(train.item_name)

In [ ]:
%%time
clf = LogisticRegression(max_iter=400)
cross_val_score(clf, X_train, train.category_id, cv=3, scoring='f1_weighted')

CPU times: user 2min 43s, sys: 1min 51s, total: 4min 34s
Wall time: 2min 29s


In [ ]:
clf.fit(X_train, train.category_id)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=400,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
cross_val_score(clf, X_train, train.category_id, cv=3, scoring='f1_weighted')

array([0.64342123, 0.62674415, 0.5955667 ])

In [ ]:
np.array([0.64342123, 0.62674415, 0.5955667 ]).mean()

0.6219106933333333

In [ ]:
import pickle
SUBM_PATH = '/content/gdrive/MyDrive/Colab_Notebooks/DFC/t1_sub/'
pickle.dump(tfidf, open(SUBM_PATH + 'tfidf', 'wb'))
pickle.dump(clf, open(SUBM_PATH + 'clf_task1', 'wb'))

#Генерация Сабмита для подачи в систему

In [ ]:
import zipfile 
compression = zipfile.ZIP_DEFLATED

submission_name = SUBM_PATH + 'submission.zip'
with zipfile.ZipFile(submission_name, 'w') as zipObj:
    for filename in [
        'clf_task1',
        'tfidf',
        'script.py',
    ]:
        zipObj.write(
            f'{SUBM_PATH}{filename}', 
            arcname=filename, 
            compress_type=compression
        )
    print(zipObj.namelist())

!ls -lh {submission_name}

['clf_task1', 'tfidf', 'script.py']
-rw------- 1 root root 1.1M Mar  9 05:36 /content/gdrive/MyDrive/Colab_Notebooks/DFC/t1_sub/submission.zip
